# Notebook to experiment with topic model

In [ ]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

import nltk

nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

from nltk.stem.wordnet import WordNetLemmatizer